In [1]:
## Libraries and setup
# !pip install diagonal_b6
import diagonal_b6 as b6
import os
import json
import geopandas as gpd
import pandas as pd


# get working directory
working_directory =  os.getcwd()

# connect to the Bedrock engine
w = b6.connect_insecure("cdtweek-2023.diagonal.works:8002")

C:\Users\b8008458\AppData\Local\Temp\ipykernel_13792\2994851335.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
## get lsoas
# get Bedrock data for Liverpool study area
liverpool = b6.cap_polygon(b6.ll(53.4127,-2.9608), 12000)

# get lsoas
lsoas = b6.find(b6.and_(b6.tagged("#boundary", "lsoa"), b6.intersecting(liverpool)))

In [21]:
## get pwc
# set filenames
#filename = "\pwc_liverpool.geojson"
filename = "/pwc_liverpool.geojson"
data_directory = working_directory.replace('Scripts', 'Data')

#  create geodataframe from file
pwc = gpd.read_file(data_directory + filename)

In [22]:
pwc

,FID,LSOA21CD,GlobalID,OBJECTID,LAD22CD,LAD22NM,BNG_E,BNG_N,LONG,LAT,GlobalID.1,geometry
0,13581,E01006745,d6c18067-7d15-4017-8b45-8074a789aa80,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.89879 53.35572)
1,13614,E01006583,17699269-164a-4b0f-a435-6780c10ccb75,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.88783 53.39107)
2,13671,E01006755,ab3177e6-a285-4bfe-9825-a75fe49f6eb1,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.83687 53.34231)
3,13683,E01006521,d5d9dcd8-61e9-4923-a3b0-2fa5e36337a6,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.93530 53.37470)
4,13704,E01034411,e0a767c8-2870-41a9-8970-4fc85c126c66,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.99355 53.40576)
...,...,...,...,...,...,...,...,...,...,...,...,...
297,20301,E01006721,d3a3e57f-dcd3-4eb6-bfc8-ee825f665011,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.92916 53.39800)
298,20310,E01006653,1037ea38-943a-4f8e-aad5-49bc151ecfb4,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.92267 53.47044)
299,20313,E01006784,2b3ea3c3-f1a1-4da8-87e5-63ce3feb3fa2,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.96736 53.46172)
300,20353,E01006775,17032931-a5f4-4f1b-9054-8071503fff22,252,E08000012,Liverpool,339361,390553,-2.91364,53.4083,9d4b2ea6-7ad6-4e9b-9c17-207627542c66,POINT (-2.86152 53.39550)


In [5]:
## set up list of coordinate pairs
pwc_coords = [(x,y) for x,y in zip(pwc['geometry'].x , pwc['geometry'].y)]

In [7]:
# Extract the number of coordinate pairs
coords = range(len(pwc_coords))

# Accessibility for adults
# For a set of amenities, calculate how many are within a 1,200 metres range from the selected population weighted centroid.
schools = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","school"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        schools.append(amenity_gdf.shape[0])
        
        
supermarkets = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#shop","supermarket"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        supermarkets.append(amenity_gdf.shape[0])
        

pharmacies = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","pharmacy"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        pharmacies.append(amenity_gdf.shape[0])


atms = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","atm"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        atms.append(amenity_gdf.shape[0])


parks = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","park"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        parks.append(amenity_gdf.shape[0])


cinemas = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","cinema"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        cinemas.append(amenity_gdf.shape[0])


gyms = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 1200 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","gym"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        gyms.append(amenity_gdf.shape[0])

index_adults = pd.DataFrame({'pwc_id':pwc.FID,'LSOA21CD':pwc.LSOA21CD,'schools':schools, 'supermarkets':supermarkets, 'pharmacies':pharmacies, 'atms':atms, 'parks':parks, 'cinemas':cinemas, 'gyms':gyms})


In [8]:
# Accessibility for seniors
# For a set of amenities, calculate how many are within a 800 metres range from the selected population weighted centroid.
schools = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","school"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        schools.append(amenity_gdf.shape[0])
        
        
supermarkets = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#shop","supermarket"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        supermarkets.append(amenity_gdf.shape[0])
        

pharmacies = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","pharmacy"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        pharmacies.append(amenity_gdf.shape[0])


atms = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","atm"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        atms.append(amenity_gdf.shape[0])


parks = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","park"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        parks.append(amenity_gdf.shape[0])


cinemas = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","cinema"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        cinemas.append(amenity_gdf.shape[0])


gyms = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 800 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","gym"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        gyms.append(amenity_gdf.shape[0])

index_seniors = pd.DataFrame({'pwc_id':pwc.FID,'LSOA21CD':pwc.LSOA21CD,'schools':schools, 'supermarkets':supermarkets, 'pharmacies':pharmacies, 'atms':atms, 'parks':parks, 'cinemas':cinemas, 'gyms':gyms})


In [9]:
# Accessibility for children
# For a set of amenities, calculate how many are within a 500 metres range from the selected population weighted centroid.
schools = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 500 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#amenity","school"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        schools.append(amenity_gdf.shape[0])

parks = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 500 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","park"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        parks.append(amenity_gdf.shape[0])


playgrounds = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 500 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","playground"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        playgrounds.append(amenity_gdf.shape[0])

pitches = []
for i in coords:
        point_x = pwc_coords[i][1]
        point_y = pwc_coords[i][0]
        
        # get area around every given set of pwc points
        temp_area = b6.cap_polygon(b6.ll(point_x, point_y)
                            , 500 # set this to your boundary distance (in meters)
                            )
        
        # get all the amenities within the area aorund the pwc points
        amenity = b6.find(b6.and_(b6.tagged("#leisure","pitch"), b6.intersecting(temp_area)))

        # save results to geojson
        amenity_geojson = w(b6.to_geojson_collection(amenity))

        # save geojson to gdf
        amenity_gdf = gpd.GeoDataFrame.from_features(amenity_geojson)
        
        # Generate a count of how many amenities can be accessed from each pwc
        pitches.append(amenity_gdf.shape[0])

index_children = pd.DataFrame({'pwc_id':pwc.FID,'LSOA21CD':pwc.LSOA21CD,'schools':schools, 'parks':parks, 'playgrounds':playgrounds, 'pitches':pitches})

In [10]:
# Calculate accessibility score adults
index_adults['score'] = index_adults[['schools','supermarkets','pharmacies', 'atms','parks','cinemas','gyms']].astype(bool).sum(axis=1)
index_adults['score_norm'] = (index_adults['score'] - index_adults['score'].min())/ (index_adults['score'].max() - index_adults['score'].min())

# Calculate accessibility score seniors
index_seniors['score'] = index_seniors[['schools','supermarkets','pharmacies', 'atms','parks','cinemas','gyms']].astype(bool).sum(axis=1)
index_seniors['score_norm'] = (index_seniors['score'] - index_seniors['score'].min())/ (index_seniors['score'].max() - index_adults['score'].min())

# Calculate accessibility score children
index_children['score'] = index_children[['schools','parks','playgrounds','pitches']].astype(bool).sum(axis=1)
index_children['score_norm'] = (index_children['score'] - index_children['score'].min())/ (index_children['score'].max() - index_children['score'].min())

In [17]:
# Test points
point_y = (-2.97995)
point_x = (53.40342)

## Find the nearest OSM point to a population weighted centriods

# Start with a 1m search radius
radius = 0.5

# Initialize an empty geodataframe to store your results
nearest_gdf = gpd.GeoDataFrame()

# Keep looping until you find at least one point
while len(nearest_gdf) == 0:
    # Find all points within the current search radius
    nearest = b6.find_points(b6.within_cap((b6.ll(point_x, point_y)), radius))
    # Convert the points to a geodataframe
    nearest_geojson = w(b6.to_geojson_collection(nearest))
    nearest_gdf = gpd.GeoDataFrame.from_features(nearest_geojson)
    # Increase the search radius for the next iteration
    radius += 0.5

# Print the final search radius and the number of points found
print(f"Search radius: {radius-1}m")
print(f"Number of points found: {len(nearest_gdf)}")

Search radius: 4.0m
Number of points found: 1


In [18]:
# access the OSM node ID
for osmid,n in w(nearest):
    print(osmid)
    node_OSM_ID = osmid

# Convert the FeatureID object to a string
node_OSM_ID_str = str(node_OSM_ID)
# Split the string by "/"
split_string = node_OSM_ID_str.split("/")
# Get the last element of the resulting list and convert it to an integer
osm_id = int(split_string[-1])
# Print the resulting OSM ID
print(osm_id)

point/openstreetmap.org/node/348147262
348147262


In [13]:
point = b6.find_point(b6.osm_node_id(osm_id))

In [14]:
print(w(point).all_tags())

[('name', 'Reconciliation'), ('tourism', 'artwork')]


In [15]:
pub = point.reachable("walk", 10000, b6.tagged("#amenity","pub"))
print(pub)

In [16]:
print(w(pub))

[]
